# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nari-labs/Dia-1.6B',
 '/sand-ai/MAGI-1',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/ostris/Flex.2-preview',
 '/moonshotai/Kimi-Audio-7B-Instruct',
 '/models',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/enzostvs/deepsite',
 '/spaces/InstantX/InstantCharacter',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/nvidia/describe-anything-model-demo',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/Anthropic/values-in-the-wild',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transfor

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nari-labs/Dia-1.6B
Updated
1 day ago
•
80.3k
•
1.4k
sand-ai/MAGI-1
Updated
1 day ago
•
473
microsoft/bitnet-b1.58-2B-4T
Updated
about 3 hours ago
•
35.9k
•
850
ostris/Flex.2-preview
Updated
3 days ago
•
4.82k
•
261
moonshotai/K

In [22]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [24]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnari-labs/Dia-1.6B\nUpdated\n1 day ago\n•\n80.3k\n•\n1.4k\nsand-ai/MAGI-1\nUpdated\n1 day ago\n•\n474\nmicrosoft/bitnet-b1.58-2B-4T\nUpdated\nabout 3 hours ago\n•\n35.9k\n•\n850\nostris/Flex.2-preview\nUpdated\n3 days ago\n•\n4.82k\n•\n262\nmoonshotai/Kimi-Audio-7B-Instruct\nUpdated\nabout 4 hours ago\n•\n1.63k\n•\n190\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n868\n868\nDia 1.6B\n👯\nGener

In [25]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


```markdown
# Welcome to Hugging Face! 🤗

## The AI Community Building the Future!

At Hugging Face, we’re not just developing cutting-edge AI; we're fostering a community that believes in the power of collaboration and friendly emojis! Whether you're here to find the latest models, explore datasets, discover AI apps, or just to drink coffee on the Squishy Couch of Innovation 🛋️, you're in the right place!

---

## Our Offerings
- **1M+ Models:** We've got more models than there are flavors of ice cream! 🍦 (Well, almost!)
- **400k+ Applications:** With more applications than a late-night mobile game download spree.
- **250k+ Datasets:** More fresh data than your favorite farmer’s market—minus the long lines!

### Trending Models This Week 📰
- **nari-labs/Dia-1.6B:** Because who doesn't want a dialogue partner with a PhD? 🤖
- **sand-ai/MAGI-1:** Magic for non-magicians! 
- **microsoft/bitnet-b1.58-2B-4T:** I mean, who needs a TARDIS when you’ve got this?
  
---

## Why Hugging Face? 🤔
- **Collaboration is Key:** Join forces with over **50,000 organizations** (and counting!)—because teamwork makes the dream work! 🥳
- **Open Source:** We share! You share! Let’s share a pizza… wait, wrong kind of sharing! 🍕
- **Build your Portfolio:** Show off your work like a proud parent at a school play! 🎭

---

## Our Culture 🎉
Life at Hugging Face is built on:
- **Inclusivity:** Everyone's invited to join our crazy AI fiesta!
- **Innovative Spirit:** We never stop thinking outside the box, except when the box is full of awesome.
- **Growth Mindset:** Like our models, we keep evolving!

---

## Join Our Team! 🚀
Are you ready to shape the future of AI? We're on the lookout for creative geniuses (and coffee lovers). Roles available:
- AI Engineers
- Data Explorers
- Imaginative Creatives (Magicians encouraged!)

**Your Future Awaits! 📫** Apply now and let’s create something spectacular together!

---

## Let’s Connect! 
Follow us on social media channels or pop by to say hi on our [Discord](#). 💬

**Ready to Explore?** 
Visit us at [Hugging Face](https://huggingface.co) and join the revolution of AI!

---

*Disclaimer: Hugging Face is not responsible for the excessive joy, caffeination, or brain expansion that might occur upon visiting our site. Side effects may include the urge to innovate, collaborate, and share freely!*
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [27]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face!
**The AI Community Building the Future**

---

## What's Hugging Face? 

If you're thinking "I could really use a dose of artificial intelligence in my life", you're in the right place! Hugging Face is like the social media platform your favorite geeky machine learning nerds have been waiting for, but instead of selfies, we’ve got models, datasets, and applications coming out of our... well, AI!

## Join the Machine Learning Revolution 🌌

At Hugging Face, we boast over **1 million models** and **250k datasets** just waiting for you to do something fabulous with them. Whether you want to chat with a robot, train a smart dog, or simply explore the mysteries of data—our playground is open 24/7! 

## Hugging Face Culture (We're All About the Love!) 🥰

Here at Hugging Face, we don’t just **“hug”**—we embrace innovation with open arms. Our team is made up of brilliant minds from around the globe who love working together to create AI magic! Team bonding activities? How about collaborative coding sprints or pizza parties where the toppings are as diverse as the models we create!

### **Why Work with Us?**
- Work with cutting-edge tech (we're basically the Avengers for AI).
- Unlimited coffee (or whatever keeps you going)!  
- Competitive salaries that won’t make your wallet cry. 💰
- Flexible hours—because we get it, ideas don’t always flow between 9-5.

## Our Customers Not "Just Bot" (Get it? 😂)

More than **50,000 organizations** trust us! From startups to giants like Apple, Amazon, and Google. We’ve got more followers than your average influencer—and they’re all about **AI**! 

### **Our Amazing Clients Include:**
- **Google** - Who constantly nibbles on our datasets 🥳
- **Microsoft** - We may have hugged them a few times! 🤗
- **Grammarly** - So your emails don't get embarrassing. 
- And many more...

---

## Careers at Hugging Face 

Ready to step into the world of AI magic? We’re always on the lookout for talented individuals who like to dream big (and code even bigger).  Check out our job section, where converting your dream job into reality is just a click away!

**Join us, and let’s create the future of AI together!**

---

If you're intrigued (or just want to laugh at our AI humor), visit our website and sign up today! We promise it’s more fun than a robot rave! (Spoiler: We've got a ton of great party trick models for that too 😉).

---

**Hugging Face: Building the future, one dataset at a time!** 

[Explore AI Apps](#) | [Join Our Community](#) | [Career Opportunities](#) 

---

*Formerly known as "The Place Where Algorithms Go to Party"* 🎉

In [29]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Where AI Hugs You Back! 🤗

Welcome to the whimsical world of Hugging Face—where artificial intelligence meets a warm greeting! Our mission? To build the AI community that’s actively crafting the futuristic, fuzzy future of technology. Buckle up; it’s going to be a heartwarming ride!

---

## **What Do We Do?** 🛠️

### Hugging Face is Like an AI Playground!

Hop into our magical marketplace where the machine learning community collaborates on:
- **1 Million+ Models**: Seriously, you could spend a lifetime scrolling and still not see them all!
- **250k+ Datasets**: Because who doesn’t love a good dataset? It’s like having an all-you-can-eat buffet for data!
- **Spaces**: Not your average basement—build & run applications to show off your creative coding skills! 🎨

---

## **Customer Love** 💖

Over **50,000 organizations** love us, including:
- 🦸‍♂️ **AI2**: Not just superheroes, but ones with 749 models!
- 🌍 **Meta**: Home to 2.12k models and endless scrolling.
- ☁️ **Amazon and Google**: Because they need help too, and who better than Hugging Face? 

(Wow, Google has 990 models. Talk about a data buffet!)

---

## **Career Opportunities** 🎉

Hugging Face isn’t a typical workplace; it’s more like a family reunion where everyone presents their coolest AI projects! We're growing, and so are our job openings. We’re looking for:
- **AI Wizards**: Craft spells with code (okay, it's just programming, but you get it).
- **Dataset Divers**: Dive deep into data oceans; a snorkel may be handy!
- **Model Makers**: Turn ideas into models that could potentially make even your cat jealous.

**Why join us?** Because you get to learn, grow, and occasionally hug AI—figuratively, of course!

---

## **Embrace the Culture** 🌈

At Hugging Face, we like to keep things cozy. Our culture encourages:
- **Collaboration**: Share ideas and models like it's Thanksgiving dinner.
- **Openness**: The only secrets we keep are our AI magic codes.
- **Fun**: Who says you can’t have a giggle while training neural networks?

---

## **Hugging Face: Our Secret Sauce** 🥫

We’re leveraging cutting-edge machines for all your high-performance AI needs—like a gourmet chef, but for algorithms! 

Whether you want enterprise solutions starting at just **$20/user/month**, or a little dabble with our compute options from **$0.60/hour**—we have something for everyone! 

---

### **Join Us & Get Involved!** 🌟

Ready to embark on an AI journey filled with innovation, laughter, and a sprinkle of hugs? Sign up & let’s embrace the future together! 

---

### **Connect with Us** 📱

🌐 [Explore Our Website](#)  
🐦 [Follow Us on Twitter](#)  
📖 [Connect on LinkedIn](#)  
🤖 [Join Our Discord](#)

**Hugging Face: Where AI is warm, welcoming, and always ready to give you a hug.** 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>